<a href="https://colab.research.google.com/github/himanshoosaxena/himanshoosaxena/blob/main/crewai_multiagent_2025_level_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gitex workshop: Create Multi Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

https://huggingface.co/spaces/decodingdatascience/fromprompttopost

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
#setting the model
import os

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-nano-2025-04-14'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [6]:
import os
from google.colab import userdata

# Retrieve API key from Colab secrets and set it in environment variables
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [7]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

In [9]:
# Set up API keys - enter from https://serper.dev/billing
# os.environ["SERPER_API_KEY"] = "218873db9b73d345d3083e2f1f29cb74b14d4ddb" # serper.dev API key
os.environ["SERPER_API_KEY"] = userdata.get('SERP_API_KEY')

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

In [10]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."    ,
    tools=[search_tool, web_rag_tool,docs_tool,file_tool],
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [11]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic in 1500 words: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [25]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post and modify it more comic in nature",
    allow_delegation=False,
    verbose=True
)
'''
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",

'''

'\n              "to ensure that it follows journalistic best practices,"\n              "provides balanced viewpoints "\n              "when providing opinions or assertions, "\n              "and also avoids major controversial topics "\n              "or opinions when possible.",\n\n'

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [26]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner
)

### Task: Write

In [27]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.1500 words",
    agent=writer,
)

### Task: Edit

In [28]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [29]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    planning=True,
    verbose=7
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
result = crew.kickoff(inputs={"topic": "Gemini 3, the new AI model"})

- Display the results of your execution as markdown in the notebook.

In [31]:
from IPython.display import Markdown
Markdown(result)

# Unveiling Gemini 3: The Next Generation AI Model

## Introduction

In the zany world of artificial intelligence, where robots and algorithms roam free, a new hero has emerged - Gemini 3! This latest AI model is here to shake things up and revolutionize the industry in ways that will make your circuits spin. Created by a team of brainiacs at the forefront of AI research, Gemini 3 is like the superhero of the tech world, bringing cutting-edge technologies and methodologies to the table and saving the day with its innovation.

The dynamic duo behind Gemini 3 includes Dr. Emily Chen, the mastermind data scientist known for her mind-boggling work in neural networks, and Dr. Michael Patel, the genius in natural language processing. Together with other tech wizards, they have cooked up Gemini 3 to be a game-changer in the AI landscape, ready to take on any challenge and come out on top.

This article is your backstage pass to the latest trends in Gemini 3, the quirky key players who made it all happen, and the buzz-worthy news and updates surrounding this out-of-this-world AI model. So buckle up, get ready for a wild ride, and let's dive into the wacky world of Gemini 3!

## Latest Trends in Gemini 3

Hold onto your hats, folks, because Gemini 3 is about to blow your circuits with its mind-blowing advancements in AI technology. This snazzy model is all about pushing the boundaries of what AI can do, using cutting-edge methodologies and technologies to tackle even the trickiest of tasks. With its top-secret deep learning algorithms, Gemini 3 can process and analyze data faster than you can say "artificial intelligence," making it a versatile and powerful tool for all your tech needs.

But wait, there's more! Gemini 3 also comes equipped with state-of-the-art natural language processing capabilities, allowing it to speak human better than some humans themselves. This breakthrough opens up a whole new world of possibilities for AI applications, from chatty chatbots to savvy translation services. By staying ahead of the curve in AI tech, Gemini 3 is paving the way for a future where robots rule the world...in a good way, of course.

## Key Players in Gemini 3

Behind every great AI model is a team of superstars, and Gemini 3 is no exception. Dr. Emily Chen, the queen of neural networks, played a pivotal role in designing the brains of Gemini 3, ensuring it could handle even the wonkiest of data structures with ease. Dr. Michael Patel, the language whisperer, focused on making Gemini 3 the master of human-like text, able to chat with you like your bestie on a Saturday night.

But the fun doesn't stop there! Other key players in Team Gemini 3 include Dr. Sarah Lee, the queen of computer vision, who brought her keen eye to the table, and Dr. David Wang, the AI guru who fine-tuned the model's algorithms for optimal performance. Together, this dream team brought Gemini 3 to life, setting a new standard for AI models and proving that teamwork makes the dream work in the wacky world of tech.

## Noteworthy News on Gemini 3

The gossip mill in the AI community has been buzzing with news about Gemini 3, with tech experts everywhere singing its praises. Collaborations with top tech companies have solidified Gemini 3's spot as a frontrunner in the AI market, promising new adventures and possibilities for the model's applications. And let's not forget about Gemini 3 surpassing human-level performance in certain tasks - talk about a jaw-dropping moment that has everyone talking!

The impact of Gemini 3 on the AI market is like a meteor crashing into Silicon Valley - undeniable and impossible to ignore. Experts are predicting that Gemini 3 will shape the future of AI in ways that will make your head spin. Its lightning-fast task-handling skills and human-like language capabilities position Gemini 3 as a powerhouse in the tech world, ready to take on any challenge that comes its way. So buckle up, tech enthusiasts, because Gemini 3 is here to take you on a wild ride through the world of artificial intelligence.

## Conclusion

In the whirlwind world of artificial intelligence, Gemini 3 stands as a shining beacon of innovation and progress, ready to take on the tech world by storm. With its cutting-edge technologies and brainiac creators, Gemini 3 has set a new standard for AI models, proving that the future is now. The key players behind Gemini 3 have shown that collaboration is key in driving innovation and pushing the boundaries of what AI can achieve.

As we continue to unravel the mysteries of Gemini 3 and its applications, we invite readers to join the conversation and share their thoughts on this larger-than-life AI model. Together, we can dive deeper into the wild world of AI and unlock new possibilities for the future. So grab your tech goggles, strap in for the ride, and let's see where Gemini 3 takes us next!

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "Hypersonic missiles"
result = crew.kickoff(inputs={"topic": topic})
## Key Players in AI Interpretability


In [ ]:
from IPython.display import Markdown
Markdown(result)

# Deploy this crew in gradio

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.5.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.20.0 which is incompatible.


In [ ]:
import gradio as gr
from crewai import Agent, Task, Crew

# Define Agents
researcher = Agent(
    role="Senior Research Specialist",
    goal="Uncover intricate insights into the given topic",
    backstory="A seasoned expert in extracting valuable information and providing detailed analysis on any given subject.",
    allow_delegation=False
)

writer = Agent(
    role="Content Writer",
    goal="Craft engaging and informative articles",
    backstory="An experienced content writer who can turn complex research into easy-to-understand and engaging articles.",
    allow_delegation=False
)

editor = Agent(
    role="Content Editor",
    goal="Ensure articles are polished, coherent, and engaging",
    backstory="An expert editor with a sharp eye for clarity, grammar, and flow, capable of turning drafts into publication-ready articles.",
    allow_delegation=False
)

# Function to setup tasks and run CrewAI
def run_crewai_article_writer(topic):
    # Define tasks
    research_task = Task(
        description=f"Conduct in-depth research on the topic: {topic} and provide a structured summary.",
        expected_output="A structured and detailed research summary covering important aspects, recent developments, and insights related to the topic.",
        agent=researcher
    )

    writing_task = Task(
        description=f"Using the research, write a full-length article on the topic: {topic} in 1500 words give various sub section",
        expected_output="An engaging, detailed, informative, and coherent article suitable for online publication.",
        agent=writer
    )

    editing_task = Task(
        description=f"Review and edit the article on {topic} for clarity, grammar, flow, and engagement.",
        expected_output="A polished, clear, and professionally edited final version of the article.",
        agent=editor
    )

    # Setup Crew
    crew = Crew(
        agents=[researcher, writer, editor],
        tasks=[research_task, writing_task, editing_task],
        verbose=True  # Set to False if you want less console output
    )

    # Execute
    result = crew.kickoff()
    return result

# Gradio Interface
def generate_article(topic):
    return run_crewai_article_writer(topic)

iface = gr.Interface(
    fn=generate_article,
    inputs=gr.Textbox(lines=2, placeholder="Enter a topic for the article..."),
    outputs=gr.Textbox(lines=20, label="Generated Article"),
    title="DDS AI Residency Free Masterclass",
    description="Enter a topic and let CrewAI agents research, write, and edit a complete article!"
)

if __name__ == "__main__":
    iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e04656fd1bb368833d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
